In [107]:
C1 = [-1, -4, 5]
C2 = [-1, 6, -5]
C3 = [-1, -6, 7]
C4 = [-1, -7, -5]
C5 = [1, 4, 6]

clauses = {
  1: C1, 
  2: C2, 
  3: C3, 
  4: C4, 
  5: C5
}

In [108]:
# Each element of the assignment history is represented as a 3-uple.
#   - Its first element is either 0 (decision) or > 0 (number of a clause)
#   - Its second element is the variable number
#   - Its third element is the value we assigned to that variable 


assignment_history = [
  (0, 1, 1),
  (0, 2, 0),
  (0, 3, 0),
  (0, 4, 1),
  (1, 5, 1),
  (2, 6, 1),
  (3, 7, 1)
]

In [109]:
def resolvent(C1, C2, common_var):
    if (common_var in C1 and -common_var in C2) or (-common_var in C1 and common_var in C2):
        R = C1.union(C2)
        R.remove(common_var)
        R.remove(-common_var)
        return R
    
    return None

In [110]:
def learning_procedure(AH, conflict_clause, clauses):
    k = len(AH)
    X = conflict_clause
    for i in range(k, 0, -1):
        C_num = AH[i-1][0]
        Q = AH[i-1][1]
        if C_num > 0 and resolvent(set(X), set(clauses[C_num]), Q) is not None:
            X = resolvent(set(X), set(clauses[C_num]), Q)

    return X

In [111]:
learning_procedure(assignment_history, clauses[4], clauses)

{-4, -1}

### Backtracking


In [112]:
# Assigning values to variables
vals = {
    1: 1,
    2: 0,
    3: 0,
    4: 1,
    5: 1,
    6: 1,
    7: 1
}

In [113]:
def is_clause_true(clause, vals):
    result = 0
    for i in clause:
        if i > 0:
            result = result or vals[i]
        else:
            result = result or not(vals[-i])

    return result


# Function that returns a conflict clause if it exists
def get_conflict_clause(vals, clauses):
    for _, clause in clauses.items():
        if not is_clause_true(clause, vals):
            return clause
    return None
    

# Function that checks whether a clause in a unit clause. 
# If so, it returns the unassigned literal in the clause.
def is_unit_clause(clause, P):
    count_unassigned = 0
    for l in clause:
        if P[abs(l)] == -1:
            count_unassigned += 1
            unassigned_literal = l
            if count_unassigned > 1: # This means that there is more than one unassigned variable
                return (False, None)
        
        elif l > 0 and P[abs(l)] == 1: # This means that it's a literal equal to 1
            return (False, None)
        
        elif l < 0 and P[abs(l)] == 0: # Also, this means that it's a literal equal to 1
            return (False, None)
        
    if count_unassigned == 1:
        return (True, unassigned_literal)
    return (False, None)

In [114]:

"""Returns 0 if UNSAT, 2 if next step is STEP 2, and 5 if next step is STEP 5"""
def step4(vals, clauses, AH):
    conflict_clause = get_conflict_clause(vals, clauses)
    if conflict_clause is None:
        # There is no conflict clause, so go to step 5
        return 5
    
    learned_clause = learning_procedure(AH, conflict_clause, clauses)

    # if the learned clause is empty, return UNSAT (0)
    if (len(learned_clause) == 0):
        return 0
    
    # else, add the learned clause to F
    # TODO: should i check if the learned clause already exists?
    clauses[len(clauses) + 1] = learned_clause

    # Then, do backtracking on the learned clause
    i = len(AH) - 1

    while not is_unit_clause(learned_clause, vals)[0]:
        var_idx = AH[i][1]
        vals[var_idx] = -1
        del AH[i]

        i -= 1

    # Then, go to step 2
    return 2

        


In [115]:
step4(vals, clauses, assignment_history)

2

In [116]:
assignment_history

[(0, 1, 1), (0, 2, 0), (0, 3, 0)]

In [117]:
clauses

{1: [-1, -4, 5],
 2: [-1, 6, -5],
 3: [-1, -6, 7],
 4: [-1, -7, -5],
 5: [1, 4, 6],
 6: {-4, -1}}

In [118]:
vals

{1: 1, 2: 0, 3: 0, 4: -1, 5: -1, 6: -1, 7: -1}